# Définissions des fonctions de base de la cryptographie et des variables globales


In [20]:
alpha = "abcdefghijklmnopqrstuvwxyz"
lettre_alpha = {i: lettre for i, lettre in enumerate(alpha)}
crypto2Message = 'atgtqcxshzzdbtdltfepfjydgitqodfwtqoitxkhfxcdzhfgitxwjxjhqqdjgtxdltfepfjchqqtgdjtqowjtdfqgtwtzjbftictltgwdqotx'

#Fonction permettant d'obtenir le code (rang) d'une lettre dans l'alphabet
def code (lettre):
    code = 0
    if len (lettre) != 1:
        return -1
    else:
        for i in range (0, len (alpha)):
            if lettre == alpha[i]:
                code = i
    return code

#Fonction permettant d'obtenir la lettre correspondant à un code (rang) dans l'alphabet
def lettre(code):
    lettre = ""
    if code < 0 or code > 25:
        return -1
    else:
        lettre = alpha[code]
    return lettre

# Fonction permettant de claculer un inverse modulaire d'un nombre n dans une ensemble Z/mZ 
def inverse_modulaire(n, m):
    """
    Calcule l'inverse modulaire de n modulo m en utilisant l'algorithme d'Euclide étendu.
    Retourne l'inverse si il existe, sinon retourne None si n et m ne sont pas premiers entre eux.
    """
    t, new_t = 0, 1
    r, new_r = m, n
    
    while new_r != 0:
        quotient = r // new_r
        t, new_t = new_t, t - quotient * new_t
        r, new_r = new_r, r - quotient * new_r

    if r > 1:
        return None  # n n'a pas d'inverse mod m
    if t < 0:
        t = t + m

    return t

## Fonction permettant de décrypté un message codé avec un chiffrement affine 

In [21]:
# permet de déchiffrer un message crypté avec une clé affine si a est inversible modulo 26
def dechiffreAffine(message_crypte, a, b):
    """
    Déchiffre un message en utilisant la clé affine.
    Args:
    - message_crypte (str): le message crypté
    - a (int): coefficient multiplicatif de la clé affine
    - b (int): décalage de la clé affine

    Returns:
    - str: le message déchiffré
    """
    message_dechiffre = ""
    inverse_a = inverse_modulaire(a, 26)
    if inverse_a is None:
        raise ValueError(f"Pas d'inverse modulaire pour a = {a}")

    for lettre in message_crypte:
        if lettre in alpha:  # Vérifie que la lettre est dans l'alphabet
            code_lettre = code(lettre)
            code_lettre_claire = (inverse_a * (code_lettre - b)) % 26
            message_dechiffre += lettre_alpha[code_lettre_claire]
        else:
            message_dechiffre += lettre  # Conserve les caractères non-alphabétiques
    return message_dechiffre

## Fonction permettant de testé 5 fois un déchiffrement affine en fonction de l'analyse fréquentielle

In [22]:
#Import de la bibliothèque permettant de réaliser l'analyse fréquentielle
from collections import Counter 
def crypto2(message_crypte):
    """
    Déchiffre un message en utilisant une boucle pour tester 5 hypothèses de lettres fréquentes en clair.
    Calcule dynamiquement a et b à chaque hypothèse.

    Args:
    - message_crypte (str): le message crypté à déchiffrer.

    Returns:
    - None: Affiche les messages déchiffrés pour chaque hypothèse.
    """
    modulo = 26  # Taille de l'alphabet utilisé dans le chiffrement affine (A-Z)

    # Analyse fréquentielle du texte crypté
    # Comptabilise les lettres dans le texte crypté et identifie les deux plus fréquentes
    c = Counter(message_crypte)  # Analyse des fréquences des lettres
    freq = c.most_common(2)  # Obtenir les deux lettres les plus fréquentes
    lettre_codee_1 = freq[0][0]  # Première lettre la plus fréquente (cryptée)
    lettre_codee_2 = freq[1][0]  # Deuxième lettre la plus fréquente (cryptée)
    code_lettre_codee_1 = code(lettre_codee_1)  # Indice numérique de la première lettre cryptée
    code_lettre_codee_2 = code(lettre_codee_2)  # Indice numérique de la deuxième lettre cryptée

    # Affichage des lettres fréquentes identifiées
    compteur = 1  # Compteur pour afficher le numéro d'essai
    print(f"Lettre codée 1 : '{lettre_codee_1}' (la plus fréquente dans le texte)")
    print(f"Lettre codée 2 : '{lettre_codee_2}' (deuxième plus fréquente)\n")

    # Lettres fréquentes en clair (hypothèses)
    # On suppose que la deuxième lettre fréquente correspond à une des lettres courantes en clair
    lettres_frequentes_en_clair = ['i', 'a', 's', 't', 'n']  # Lettres claires les plus fréquentes

    # Boucle pour tester les 5 hypothèses
    for lettre_claire in lettres_frequentes_en_clair:
        print(f"ESSAI {compteur}")
        print(f"Test de l'hypothèse : '{lettre_claire}' comme 2e lettre fréquente en clair")
        compteur += 1  # Incrémente le compteur pour chaque essai

        # Calcul des différences entre les indices des lettres codées et claires
        delta_code = (code_lettre_codee_2 - code_lettre_codee_1) % modulo  # Différence cryptée
        diff_clair = (code(lettre_claire) - code('e')) % modulo  # Différence claire
        inverse_diff_clair = inverse_modulaire(diff_clair, modulo)  # Inverse modulaire de la différence claire

        # Si l'inverse modulaire n'existe pas, l'hypothèse est ignorée
        if inverse_diff_clair is None:
            print(f"Pas d'inverse modulaire pour la différence {diff_clair}. Hypothèse ignorée.\n")
            continue

        # Calcul dynamique de a
        # a est déterminé par la différence cryptée et l'inverse modulaire de la différence claire
        a = (delta_code * inverse_diff_clair) % modulo

        # Calcul de b
        # b est calculé à partir de la première lettre codée et claire, et de la clé a
        b = (code_lettre_codee_1 - a * code('e')) % modulo

        # Vérification de la validité de a
        # Si a n'a pas d'inverse modulaire, la clé est invalide et l'hypothèse est ignorée
        if inverse_modulaire(a, modulo) is None:
            print(f"Clé invalide : a = {a}, b = {b}. Hypothèse ignorée.\n")
            continue

        # Déchiffrement du message
        # Applique les clés calculées (a, b) pour déchiffrer le message crypté
        message_dechiffre = dechiffreAffine(message_crypte, a, b)
        if message_dechiffre is None:  # Si le déchiffrement échoue, on passe à l'hypothèse suivante
            print(f"Impossible de déchiffrer avec a = {a}, b = {b}\n")
            continue

        # Affiche les clés trouvées et le message déchiffré
        print(f"Clés trouvées : a = {a}, b = {b}")
        print(f"Message déchiffré :\n{message_dechiffre}\n")



## Exemple d'utilisation de la fonction crypto2

In [24]:
#Déchiffrement du message crypté
crypto2(crypto2Message)

Lettre codée 1 : 't' (la plus fréquente dans le texte)
Lettre codée 2 : 'q' (deuxième plus fréquente)

ESSAI 1
Test de l'hypothèse : 'i' comme 2e lettre fréquente en clair
Pas d'inverse modulaire pour la différence 4. Hypothèse ignorée.

ESSAI 2
Test de l'hypothèse : 'a' comme 2e lettre fréquente en clair
Pas d'inverse modulaire pour la différence 22. Hypothèse ignorée.

ESSAI 3
Test de l'hypothèse : 's' comme 2e lettre fréquente en clair
Pas d'inverse modulaire pour la différence 14. Hypothèse ignorée.

ESSAI 4
Test de l'hypothèse : 't' comme 2e lettre fréquente en clair
Clés trouvées : a = 5, b = 25
Message déchiffré :
veretlkjmaagqegsewbywcfgrhetdgwpetdhekxmwklgamwrhekpckcmttgcrekgsewbywclmttergcetdpcegwtrepeacqwehleserpgtdek

ESSAI 5
Test de l'hypothèse : 'n' comme 2e lettre fréquente en clair
Clés trouvées : a = 17, b = 3
Message déchiffré :
jerendshommageaceuxquiparlentauventlesfousdamourlesvisionnairesaceuxquidonneraientvieaunrevemigueldecervantes

